In [2]:
# Import libraries
import pandas as pd
import numpy as np





In [3]:
pd.read_excel?

In [4]:
MAXIMO = pd.read_excel(r'C:\Users\rvcmalalis\OneDrive - Global Business Power\Documents\Python_Data Manipulation\Filtering Invalid Payables_ClosedMAx.xlsx'
                      ,sheet_name = 'Maximo')

In [5]:
MAXIMO.head()

,COMPSTATUSID,CHANGEBY,Format_Date,Ref_Date,CHANGEDATE,COMPANY,MEMO,ORGID,STATUS,ROWSTAMP,VLOOKUP
0,1326,MAXADMIN,2014-12-27,2014-12-27,2014-12-27-12.46.31.246000,1615,NaN,GBPC,NaN,2320001,CLOSED
1,16550,EDECASTRO,2015-08-12,2015-08-12,2015-08-12-10.39.35.960000,1615,CONDITIONAL SUSP 2015 (DID NOT SUBMIT UPDATED ...,GBPC,SUSP,33228557,CLOSED
2,16612,EDECASTRO,2015-08-17,2015-08-17,2015-08-17-15.17.39.717000,1615,APPROVAL VALIDITY JUNE 2016 / CONDITIONAL SUSP...,GBPC,APPR,33771273,CLOSED
3,18206,GBP_ACCR_OFF,2016-09-23,2016-09-23,2016-09-23-09.26.55.144000,1615,CONDITIONAL SUSPENSION 2016 / DID NOT UPDATE A...,GBPC,SUSP,87698816,CLOSED
4,18849,GBP_ACCR_OFF,2017-03-14,2017-03-14,2017-03-14-10.23.17.728000,1615,REQUEST TO LIFT SUSPENSION / UPDATED DOCUMENTS...,GBPC,ENDORSED,115900455,CLOSED


In [6]:
# Sort order bby Dat Descending per COMPANY

M_T = MAXIMO.sort_values(by = ['COMPANY','Format_Date'], ascending = True )

# Select the Columns needed

M_T = M_T[['CHANGEBY'           
           ,'COMPANY'
           ,'MEMO'
           ,'Format_Date'
           ,'STATUS']]

# Lag the Variables

M_T['Date_Lagged'] = M_T.groupby(['COMPANY'])['Format_Date'].shift(-1)
M_T['Status_Lagged'] = M_T.groupby(['COMPANY'])['STATUS'].shift(-1)


In [7]:
# Minus one day for the Status Lagged \ Don't minus 1 if the next Date Entry is same date
M_T['Date_Lagged'] = np.where(M_T['Date_Lagged'] != M_T['Format_Date']
                              ,M_T['Date_Lagged'] -  pd.to_timedelta(1, unit='d')
                              ,M_T['Date_Lagged']
                             )

In [8]:
M_T[M_T['COMPANY'] == 10005450]

,CHANGEBY,COMPANY,MEMO,Format_Date,STATUS,Date_Lagged,Status_Lagged
579,GBP_ACCR_OFF,10005450,NaN,2021-01-04,NaN,2021-01-04,TEMP
580,JJACOSALEM,10005450,NaN,2021-01-05,TEMP,2021-02-04,CLOSED
581,JJACOSALEM,10005450,NaN,2021-02-05,CLOSED,NaT,NaN


In [9]:
#Add a column that identifies Valid/Not Valid Periods [format_date,Date_Lagged]
M_T['Validity'] = np.where(M_T['STATUS'].isin(['CLOSED','SUSP']) #Status at start is Closed/Susp 
                      & np.logical_not(M_T['Status_Lagged'].isin(['CLOSED','SUSP'])) #Status at end is Closed/Susp
                                      ,'Not Valid', 'Valid') 
                     

In [10]:
M_T.head(50)

,CHANGEBY,COMPANY,MEMO,Format_Date,STATUS,Date_Lagged,Status_Lagged,Validity
0,MAXADMIN,1615,NaN,2014-12-27,NaN,2015-08-11,SUSP,Valid
1,EDECASTRO,1615,CONDITIONAL SUSP 2015 (DID NOT SUBMIT UPDATED ...,2015-08-12,SUSP,2015-08-16,APPR,Not Valid
2,EDECASTRO,1615,APPROVAL VALIDITY JUNE 2016 / CONDITIONAL SUSP...,2015-08-17,APPR,2016-09-22,SUSP,Valid
3,GBP_ACCR_OFF,1615,CONDITIONAL SUSPENSION 2016 / DID NOT UPDATE A...,2016-09-23,SUSP,2017-03-13,ENDORSED,Not Valid
4,GBP_ACCR_OFF,1615,REQUEST TO LIFT SUSPENSION / UPDATED DOCUMENTS...,2017-03-14,ENDORSED,2017-03-14,APPR,Valid
5,JJACOSALEM,1615,REQUEST TO LIFT SUSPENSION / UPDATED DOCUMENTS...,2017-03-15,APPR,2017-09-12,SUSP,Valid
6,GBP_ACCR_OFF,1615,SUSPENDED ON AUGUST 2017 DUE TO NON SUBMISSION...,2017-09-13,SUSP,2017-11-07,ENDORSED,Not Valid
7,GBP_ACCR_OFF,1615,"JUNE 2018 VALIDITY, RECOMMENDING TO LIFT SUSPE...",2017-11-08,ENDORSED,2017-11-08,APPR,Valid
8,JJACOSALEM,1615,"JUNE 2018 VALIDITY, RECOMMENDING TO LIFT SUSPE...",2017-11-08,APPR,2018-10-12,CONDITIONAL,Valid
9,GBP_ACCR_OFF,1615,SUSPENDED ON OCTOBER 2018 DUE TO NON SUBMISSIO...,2018-10-13,CONDITIONAL,2018-11-14,ENDORSED,Valid


In [11]:
NETSUITE = pd.read_excel(r'C:\Users\rvcmalalis\OneDrive - Global Business Power\Documents\Python_Data Manipulation\Filtering Invalid Payables_ClosedMAx_v2.xlsx'
                      ,sheet_name = 'NS')

In [12]:
NETSUITE.shape

(246, 25)

In [13]:
print(NETSUITE.info())
print(M_T.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 246 entries, 0 to 245
Data columns (total 25 columns):
 #   Column                           Non-Null Count  Dtype         
---  ------                           --------------  -----         
 0   Vendor No.                       246 non-null    int64         
 1   Test                             246 non-null    object        
 2   Company                          246 non-null    object        
 3   Type                             246 non-null    object        
 4   Date                             246 non-null    datetime64[ns]
 5   Transaction Number               246 non-null    object        
 6   Document Number                  246 non-null    object        
 7   Batch Name                       246 non-null    object        
 8   Vendor_Customer Name             246 non-null    object        
 9   PO Number                        150 non-null    object        
 10  PO Date                          150 non-null    datetime64[ns

In [14]:
NETSUITE.head(50)

,Vendor No.,Test,Company,Type,Date,Transaction Number,Document Number,Batch Name,Vendor_Customer Name,PO Number,...,Account (Line): Name,Amount,Line Description,Withholding Tax Code,Plant /Site,Units/Profit Center,Transaction Status: Description,JV Ref Doc,Start of Billing Period,End of Billing Period
0,1615,"GEO-TRANSPORT & CONSTRUCTION,INC.",CEDC,Accounts Payable,2021-01-25,APV-CEDC-000009125,SOA-7261,RENTAL OF FLAT BED TRUCK FOR CCWP 1C MOTOR BEA...,"1615 GEO-TRANSPORT & CONSTRUCTION,INC.",PO-CEDC-000003168,...,Accounts Payable - Suppliers,-47767.86,RENTAL OF FLAT BED TRUCK FOR CCWP 1C MOTOR BEA...,NaN,NaN,NaN,Paid In Full,NaN,NaN,NaN
1,1615,"GEO-TRANSPORT & CONSTRUCTION,INC.",CEDC,Accounts Payable,2021-01-26,APV-CEDC-000009143,SOA-7285,RENTAL OF FLAT BED TRUCK FOR CCWP 1C MOTOR BEA...,"1615 GEO-TRANSPORT & CONSTRUCTION,INC.",PO-CEDC-000003246,...,Accounts Payable - Suppliers,-47767.86,RENTAL OF FLAT BED TRUCK FOR CCWP 1C MOTOR BEA...,NaN,NaN,NaN,Paid In Full,NaN,NaN,NaN
2,1615,"GEO-TRANSPORT & CONSTRUCTION,INC.",CEDC,Accounts Payable,2021-01-25,APV-CEDC-000009125,SOA-7261,RENTAL OF FLAT BED TRUCK FOR CCWP 1C MOTOR BEA...,"1615 GEO-TRANSPORT & CONSTRUCTION,INC.",PO-CEDC-000003168,...,Accounts Payable - Suppliers,-47767.86,RENTAL OF FLAT BED TRUCK FOR CCWP 1C MOTOR BEA...,NaN,CEDC : TOL:CEDC : TOL,Common,Paid In Full,NaN,NaN,NaN
3,1615,"GEO-TRANSPORT & CONSTRUCTION,INC.",CEDC,Accounts Payable,2021-01-26,APV-CEDC-000009143,SOA-7285,RENTAL OF FLAT BED TRUCK FOR CCWP 1C MOTOR BEA...,"1615 GEO-TRANSPORT & CONSTRUCTION,INC.",PO-CEDC-000003246,...,Accounts Payable - Suppliers,-47767.86,RENTAL OF FLAT BED TRUCK FOR CCWP 1C MOTOR BEA...,NaN,CEDC : TOL:CEDC : TOL,Common,Paid In Full,NaN,NaN,NaN
4,1615,"GEO-TRANSPORT & CONSTRUCTION,INC.",TPC,Accounts Payable,2020-08-27,APV-TPC-000004490,SOA#7042,C100|DS|SOA#7042|PO-TPC-000003128|T20-PO-3727|...,"1615 GEO-TRANSPORT & CONSTRUCTION,INC.",PO-TPC-000003128,...,Accounts Payable - Suppliers,-55983.93,C100|DS|SOA#7042|PO-TPC-000003128|T20-PO-3727|...,NaN,TPC1 : SANGI:TPC1 : SANGI,U 1,Paid In Full,NaN,NaN,NaN
5,1724,CLC MARKETING VENTURES CORP.,PEDC,Accounts Payable,2020-08-06,APV-PEDC-000001692,SI No. 0544,C120|DS|SI#0544|PE20-PO-2031|Common-U3|1lot PU...,1724 CLC MARKETING VENTURES CORP.,PO-PEDC-000003579,...,Accounts Payable - Suppliers,-781220.49,C120|DS|SI#0544|PE20-PO-2031|Common-U3|1lot PU...,NaN,PE : Ilo2:PE : Ilo2,Common,Paid In Full,NaN,NaN,NaN
6,1724,CLC MARKETING VENTURES CORP.,PEDC,Accounts Payable,2020-08-06,APV-PEDC-000001693,SI No. 0545,C120|DS|SI#0545|PE20-PO-2030|Common-U3|1 lot P...,1724 CLC MARKETING VENTURES CORP.,PO-PEDC-000003578,...,Accounts Payable - Suppliers,-417410.71,C120|DS|SI#0545|PE20-PO-2030|Common-U3|1 lot P...,NaN,PE : Ilo2:PE : Ilo2,Common,Paid In Full,NaN,NaN,NaN
7,1724,CLC MARKETING VENTURES CORP.,PEDC,Accounts Payable,2020-08-06,APV-PEDC-000001691,SI No. 0543,C120|DS|SI#0543|PE20-PO-2032|Common-U3|1lot PU...,1724 CLC MARKETING VENTURES CORP.,PO-PEDC-000003580,...,Accounts Payable - Suppliers,-1593063.41,C120|DS|SI#0543|PE20-PO-2032|Common-U3|1lot PU...,NaN,PE : Ilo2:PE : Ilo2,Common,Paid In Full,NaN,NaN,NaN
8,10000060,"Schema Konsult, Inc.",PEDC,Accounts Payable,2020-06-08,APV-PEDC-000001033,SI B19-495,C011|DS|SI B19-495|Other Prof. Services-Non O&...,"10000060 Schema Konsult, Inc.",NaN,...,Accounts Payable - Suppliers,-216517.85,C011|DS|SI B19-495|Other Prof. Services-Non O&...,NaN,PE : Ilo1:PE : Ilo1,U 1,Paid In Full,NaN,NaN,NaN
9,10000159,MIKRON POWER CONTROLS CORPORATION,PEDC,Accounts Payable,2020-08-31,APV-PEDC-000001951,SI No. 8809,C158|DGO|SI#8809|PE20-PO-1621|Common-U3|1pc p...,10000159 MIKRON POWER CONTROLS CORPORATION,PO-PEDC-000003180,...,Accounts Payable - Suppliers,-19722.32,C158|DGO|SI#8809|PE20-PO-1621|Common-U3|1pc p...,NaN,PE : Ilo2:PE : Ilo2,Common,Paid In Full,NaN,NaN,NaN


In [181]:
#pd.merge?

In [16]:
# Merge the two dataset


DF_Merge = pd.merge(NETSUITE, M_T
                    ,how = 'left'
                    ,left_on = 'Vendor No.'
                    ,right_on = 'COMPANY' )
DF_Merge.head(10)

,Vendor No.,Test,Company,Type,Date,Transaction Number,Document Number,Batch Name,Vendor_Customer Name,PO Number,...,Start of Billing Period,End of Billing Period,CHANGEBY,COMPANY,MEMO,Format_Date,STATUS,Date_Lagged,Status_Lagged,Validity
0,1615,"GEO-TRANSPORT & CONSTRUCTION,INC.",CEDC,Accounts Payable,2021-01-25,APV-CEDC-000009125,SOA-7261,RENTAL OF FLAT BED TRUCK FOR CCWP 1C MOTOR BEA...,"1615 GEO-TRANSPORT & CONSTRUCTION,INC.",PO-CEDC-000003168,...,NaN,NaN,MAXADMIN,1615,NaN,2014-12-27,NaN,2015-08-11,SUSP,Valid
1,1615,"GEO-TRANSPORT & CONSTRUCTION,INC.",CEDC,Accounts Payable,2021-01-25,APV-CEDC-000009125,SOA-7261,RENTAL OF FLAT BED TRUCK FOR CCWP 1C MOTOR BEA...,"1615 GEO-TRANSPORT & CONSTRUCTION,INC.",PO-CEDC-000003168,...,NaN,NaN,EDECASTRO,1615,CONDITIONAL SUSP 2015 (DID NOT SUBMIT UPDATED ...,2015-08-12,SUSP,2015-08-16,APPR,Not Valid
2,1615,"GEO-TRANSPORT & CONSTRUCTION,INC.",CEDC,Accounts Payable,2021-01-25,APV-CEDC-000009125,SOA-7261,RENTAL OF FLAT BED TRUCK FOR CCWP 1C MOTOR BEA...,"1615 GEO-TRANSPORT & CONSTRUCTION,INC.",PO-CEDC-000003168,...,NaN,NaN,EDECASTRO,1615,APPROVAL VALIDITY JUNE 2016 / CONDITIONAL SUSP...,2015-08-17,APPR,2016-09-22,SUSP,Valid
3,1615,"GEO-TRANSPORT & CONSTRUCTION,INC.",CEDC,Accounts Payable,2021-01-25,APV-CEDC-000009125,SOA-7261,RENTAL OF FLAT BED TRUCK FOR CCWP 1C MOTOR BEA...,"1615 GEO-TRANSPORT & CONSTRUCTION,INC.",PO-CEDC-000003168,...,NaN,NaN,GBP_ACCR_OFF,1615,CONDITIONAL SUSPENSION 2016 / DID NOT UPDATE A...,2016-09-23,SUSP,2017-03-13,ENDORSED,Not Valid
4,1615,"GEO-TRANSPORT & CONSTRUCTION,INC.",CEDC,Accounts Payable,2021-01-25,APV-CEDC-000009125,SOA-7261,RENTAL OF FLAT BED TRUCK FOR CCWP 1C MOTOR BEA...,"1615 GEO-TRANSPORT & CONSTRUCTION,INC.",PO-CEDC-000003168,...,NaN,NaN,GBP_ACCR_OFF,1615,REQUEST TO LIFT SUSPENSION / UPDATED DOCUMENTS...,2017-03-14,ENDORSED,2017-03-14,APPR,Valid
5,1615,"GEO-TRANSPORT & CONSTRUCTION,INC.",CEDC,Accounts Payable,2021-01-25,APV-CEDC-000009125,SOA-7261,RENTAL OF FLAT BED TRUCK FOR CCWP 1C MOTOR BEA...,"1615 GEO-TRANSPORT & CONSTRUCTION,INC.",PO-CEDC-000003168,...,NaN,NaN,JJACOSALEM,1615,REQUEST TO LIFT SUSPENSION / UPDATED DOCUMENTS...,2017-03-15,APPR,2017-09-12,SUSP,Valid
6,1615,"GEO-TRANSPORT & CONSTRUCTION,INC.",CEDC,Accounts Payable,2021-01-25,APV-CEDC-000009125,SOA-7261,RENTAL OF FLAT BED TRUCK FOR CCWP 1C MOTOR BEA...,"1615 GEO-TRANSPORT & CONSTRUCTION,INC.",PO-CEDC-000003168,...,NaN,NaN,GBP_ACCR_OFF,1615,SUSPENDED ON AUGUST 2017 DUE TO NON SUBMISSION...,2017-09-13,SUSP,2017-11-07,ENDORSED,Not Valid
7,1615,"GEO-TRANSPORT & CONSTRUCTION,INC.",CEDC,Accounts Payable,2021-01-25,APV-CEDC-000009125,SOA-7261,RENTAL OF FLAT BED TRUCK FOR CCWP 1C MOTOR BEA...,"1615 GEO-TRANSPORT & CONSTRUCTION,INC.",PO-CEDC-000003168,...,NaN,NaN,GBP_ACCR_OFF,1615,"JUNE 2018 VALIDITY, RECOMMENDING TO LIFT SUSPE...",2017-11-08,ENDORSED,2017-11-08,APPR,Valid
8,1615,"GEO-TRANSPORT & CONSTRUCTION,INC.",CEDC,Accounts Payable,2021-01-25,APV-CEDC-000009125,SOA-7261,RENTAL OF FLAT BED TRUCK FOR CCWP 1C MOTOR BEA...,"1615 GEO-TRANSPORT & CONSTRUCTION,INC.",PO-CEDC-000003168,...,NaN,NaN,JJACOSALEM,1615,"JUNE 2018 VALIDITY, RECOMMENDING TO LIFT SUSPE...",2017-11-08,APPR,2018-10-12,CONDITIONAL,Valid
9,1615,"GEO-TRANSPORT & CONSTRUCTION,INC.",CEDC,Accounts Payable,2021-01-25,APV-CEDC-000009125,SOA-7261,RENTAL OF FLAT BED TRUCK FOR CCWP 1C MOTOR BEA...,"1615 GEO-TRANSPORT & CONSTRUCTION,INC.",PO-CEDC-000003168,...,NaN,NaN,GBP_ACCR_OFF,1615,SUSPENDED ON OCTOBER 2018 DUE TO NON SUBMISSIO...,2018-10-13,CONDITIONAL,2018-11-14,ENDORSED,Valid


In [17]:
DF_Merge_Output = DF_Merge[ ((DF_Merge['PO Date'] >= DF_Merge['Format_Date']) 
                               & (DF_Merge['PO Date'] <= DF_Merge['Date_Lagged']))
                              | 
                            ((DF_Merge['PO Date'] >= DF_Merge['Format_Date'])
                               & DF_Merge['Date_Lagged'].isnull())
                               
                               #if Transaction Date is Beyond the Last Status Date
                          
                          ]
DF_Merge_Output.head(50)

,Vendor No.,Test,Company,Type,Date,Transaction Number,Document Number,Batch Name,Vendor_Customer Name,PO Number,...,Start of Billing Period,End of Billing Period,CHANGEBY,COMPANY,MEMO,Format_Date,STATUS,Date_Lagged,Status_Lagged,Validity
15,1615,"GEO-TRANSPORT & CONSTRUCTION,INC.",CEDC,Accounts Payable,2021-01-25,APV-CEDC-000009125,SOA-7261,RENTAL OF FLAT BED TRUCK FOR CCWP 1C MOTOR BEA...,"1615 GEO-TRANSPORT & CONSTRUCTION,INC.",PO-CEDC-000003168,...,NaN,NaN,JJACOSALEM,1615,JUNE 2019 VALIDITY / REQUEST TO LIFT SUSPENSIO...,2020-12-29,APPR,2021-01-13,CLOSED,Valid
36,1615,"GEO-TRANSPORT & CONSTRUCTION,INC.",CEDC,Accounts Payable,2021-01-26,APV-CEDC-000009143,SOA-7285,RENTAL OF FLAT BED TRUCK FOR CCWP 1C MOTOR BEA...,"1615 GEO-TRANSPORT & CONSTRUCTION,INC.",PO-CEDC-000003246,...,NaN,NaN,JJACOSALEM,1615,JUNE 2019 VALIDITY / REQUEST TO LIFT SUSPENSIO...,2021-01-20,APPR,2021-02-04,CLOSED,Valid
53,1615,"GEO-TRANSPORT & CONSTRUCTION,INC.",CEDC,Accounts Payable,2021-01-25,APV-CEDC-000009125,SOA-7261,RENTAL OF FLAT BED TRUCK FOR CCWP 1C MOTOR BEA...,"1615 GEO-TRANSPORT & CONSTRUCTION,INC.",PO-CEDC-000003168,...,NaN,NaN,JJACOSALEM,1615,JUNE 2019 VALIDITY / REQUEST TO LIFT SUSPENSIO...,2020-12-29,APPR,2021-01-13,CLOSED,Valid
74,1615,"GEO-TRANSPORT & CONSTRUCTION,INC.",CEDC,Accounts Payable,2021-01-26,APV-CEDC-000009143,SOA-7285,RENTAL OF FLAT BED TRUCK FOR CCWP 1C MOTOR BEA...,"1615 GEO-TRANSPORT & CONSTRUCTION,INC.",PO-CEDC-000003246,...,NaN,NaN,JJACOSALEM,1615,JUNE 2019 VALIDITY / REQUEST TO LIFT SUSPENSIO...,2021-01-20,APPR,2021-02-04,CLOSED,Valid
89,1615,"GEO-TRANSPORT & CONSTRUCTION,INC.",TPC,Accounts Payable,2020-08-27,APV-TPC-000004490,SOA#7042,C100|DS|SOA#7042|PO-TPC-000003128|T20-PO-3727|...,"1615 GEO-TRANSPORT & CONSTRUCTION,INC.",PO-TPC-000003128,...,NaN,NaN,JJACOSALEM,1615,JUNE 2019 VALIDITY / REQUEST TO LIFT SUSPENSIO...,2020-02-21,APPR,2020-12-19,CLOSED,Valid
101,1724,CLC MARKETING VENTURES CORP.,PEDC,Accounts Payable,2020-08-06,APV-PEDC-000001692,SI No. 0544,C120|DS|SI#0544|PE20-PO-2031|Common-U3|1lot PU...,1724 CLC MARKETING VENTURES CORP.,PO-PEDC-000003579,...,NaN,NaN,JJACOSALEM,1724,SUSPENDED ON OCTOBER 2018 DUE TO NON SUBMISSIO...,2019-05-29,APPR,2020-12-19,CLOSED,Valid
109,1724,CLC MARKETING VENTURES CORP.,PEDC,Accounts Payable,2020-08-06,APV-PEDC-000001693,SI No. 0545,C120|DS|SI#0545|PE20-PO-2030|Common-U3|1 lot P...,1724 CLC MARKETING VENTURES CORP.,PO-PEDC-000003578,...,NaN,NaN,JJACOSALEM,1724,SUSPENDED ON OCTOBER 2018 DUE TO NON SUBMISSIO...,2019-05-29,APPR,2020-12-19,CLOSED,Valid
117,1724,CLC MARKETING VENTURES CORP.,PEDC,Accounts Payable,2020-08-06,APV-PEDC-000001691,SI No. 0543,C120|DS|SI#0543|PE20-PO-2032|Common-U3|1lot PU...,1724 CLC MARKETING VENTURES CORP.,PO-PEDC-000003580,...,NaN,NaN,JJACOSALEM,1724,SUSPENDED ON OCTOBER 2018 DUE TO NON SUBMISSIO...,2019-05-29,APPR,2020-12-19,CLOSED,Valid
134,10000159,MIKRON POWER CONTROLS CORPORATION,PEDC,Accounts Payable,2020-08-31,APV-PEDC-000001951,SI No. 8809,C158|DGO|SI#8809|PE20-PO-1621|Common-U3|1pc p...,10000159 MIKRON POWER CONTROLS CORPORATION,PO-PEDC-000003180,...,NaN,NaN,JJACOSALEM,10000159,SUSPENDED ON OCTOBER 2018 DUE TO NON SUBMISSIO...,2019-01-02,APPR,2020-12-13,CLOSED,Valid
143,10000159,MIKRON POWER CONTROLS CORPORATION,PEDC,Accounts Payable,2020-09-10,APV-PEDC-000002055,SI 8856,C158|DGO|SI 8856|PE19-PO-2912(PO-PEDC-00000236...,10000159 MIKRON POWER CONTROLS CORPORATION,PO-PEDC-000002368,...,NaN,NaN,JJACOSALEM,10000159,SUSPENDED ON OCTOBER 2018 DUE TO NON SUBMISSIO...,2019-01-02,APPR,2020-12-13,CLOSED,Valid


In [22]:
DF_Merge_Output[DF_Merge_Output['Validity'] != 'Valid'] #Check output

,Vendor No.,Test,Company,Type,Date,Transaction Number,Document Number,Batch Name,Vendor_Customer Name,PO Number,...,Start of Billing Period,End of Billing Period,CHANGEBY,COMPANY,MEMO,Format_Date,STATUS,Date_Lagged,Status_Lagged,Validity
1677,10005407,FLEXIBLE AUTOMATION SYSTEM CORPORATION,CEDC,Accounts Payable,2020-11-16,APV-CEDC-000008187,SI-3092,PURCHASE OF HUMAN MACHINE INTERFACE (HMI) FOR ...,10005407 FLEXIBLE AUTOMATION SYSTEM CORPORATION,PO-CEDC-000002516,...,NaN,NaN,JJACOSALEM,10005407,NaN,2020-06-08,CLOSED,2020-06-23,TEMP,Not Valid


In [ ]:
M_T[DF['COMPANY'] == 10005450] #Check output

In [185]:
# CHeck if there are duplicates
DF_Merge_Output.shape

(232, 32)

In [23]:
DF_Merge_Output.to_excel(r'C:\Users\rvcmalalis\OneDrive - Global Business Power\Documents\Python_Data Manipulation\AnalyticalDataSetv2.xlsx')